In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

# Choose an LLM Model

In [2]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Create your tools

In [3]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [9]:
@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    return a / b

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

print(divide.name)
print(divide.description)
print(divide.args)  

print(add.name)
print(add.description)
print(add.args)    

divide
Divide two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}
add
Add two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [4]:
tools_list = [multiply]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [10]:
tools_list.append(divide)
tools_list.append(add)
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

# Bind tools with LLMs

In [15]:
# we create a tool calling Agent by binding a list of tools to the llm

llm_with_tools = llm.bind_tools(tools_list)

In [6]:

response = llm_with_tools.invoke("What is 10 times 40?")

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'fyw8wfm61', 'function': {'arguments': '{"a":10,"b":40}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 220, 'total_tokens': 236, 'completion_time': 0.033715171, 'prompt_time': 0.031424749, 'queue_time': 0.046933091, 'total_time': 0.06513992}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8dc58a59-7ff0-4519-86bc-5316f3f7e4cc-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 40}, 'id': 'fyw8wfm61', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 16, 'total_tokens': 236})

In [44]:
response = llm_with_tools.invoke("What is 20 squared?")

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '9tmmc92y5', 'function': {'arguments': '{"a":20,"b":20}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 322, 'total_tokens': 341, 'completion_time': 0.050256413, 'prompt_time': 0.031179353, 'queue_time': 0.046539077, 'total_time': 0.081435766}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--259025f5-a0be-4065-9992-6474a3603999-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20, 'b': 20}, 'id': '9tmmc92y5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 322, 'output_tokens': 19, 'total_tokens': 341})

In [33]:
# Exercise: Modify the prompt in various ways

response = llm_with_tools.invoke("What is 20 divided by 5 times 100?")

response

AIMessage(content='<function=multiply({"a": 100, "b": <function=divide({"a": 20, "b": 5})})</function>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 328, 'total_tokens': 362, 'completion_time': 0.0983404, 'prompt_time': 0.037808391, 'queue_time': 0.051739169, 'total_time': 0.136148791}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8b41535c-ebe6-4694-8c3e-6917f1f2d879-0', usage_metadata={'input_tokens': 328, 'output_tokens': 34, 'total_tokens': 362})

# Tool Invokation

Tool calling is NOT tool execution

In [45]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 20, 'b': 20},
  'id': '9tmmc92y5',
  'type': 'tool_call'}]

In [46]:
tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0])

ToolMessage(content='400.0', name='multiply', tool_call_id='9tmmc92y5')